# Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# Import data

In [2]:
path = r'/Users/puchater/Documents/CareerFoundry/Data Immersion/4. Python/17042023 Instacart Bascet Analysis'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_grouping_data.pkl'))

In [4]:
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

# Data wrangling

In [5]:
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
customers.shape

(206209, 10)

In [7]:
# Renaming columns
customers.rename(columns = {'n_dependants' : 'Children'}, inplace = True)
customers.rename(columns = {'fam_status' : 'Family_status'}, inplace = True)

In [8]:
# Droping columns
customers=customers.drop(columns = ['First Name', 'Surnam'])

In [9]:
customers.head()

,user_id,Gender,STATE,Age,date_joined,Children,Family_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [10]:
# Searching for missing values
customers['user_id'],customers['income'],customers['Age'],customers['Children'] .value_counts(dropna = False)

(0          26711
 1          33890
 2          65803
 3         125935
 4         130797
            ...  
 206204    168073
 206205     49635
 206206    135902
 206207     81095
 206208     80148
 Name: user_id, Length: 206209, dtype: int64,
 0         165665
 1          59285
 2          99568
 3          42049
 4          40374
            ...  
 206204    148828
 206205    168639
 206206     53374
 206207     99799
 206208     57095
 Name: income, Length: 206209, dtype: int64,
 0         48
 1         36
 2         35
 3         40
 4         26
           ..
 206204    44
 206205    62
 206206    66
 206207    27
 206208    55
 Name: Age, Length: 206209, dtype: int64,
 0    51602
 3    51594
 1    51531
 2    51482
 Name: Children, dtype: int64)

In [11]:
# Searching for duplicates
customers_dups = customers[customers.duplicated()]

In [12]:
customers_dups

,user_id,Gender,STATE,Age,date_joined,Children,Family_status,income


In [13]:
# Checking data-type
type('user_id')

str

In [14]:
type('Age')

str

In [15]:
type('income')

str

In [16]:
type('Family_status')

str

In [17]:
type('Children')

str

In [18]:
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

In [19]:
# Changing data-type
customers['Age'] = customers['Age'].astype('int')

In [20]:
customers['user_id'] = customers['user_id'].astype('int')

In [21]:
customers['income'] = customers['income'].astype('int')

In [22]:
customers['Children'] = customers['Children'].astype('int')

# Combining data sets

In [23]:
ords_prods_merge.head(100)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,prices,price_range_loc,bussiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_order,cost_flag,order_frequency,order_frequency_flag
0,2539329,1,1,2,8,NaN,True,196,1,0.0,...,9.0,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,False,196,1,1.0,...,9.0,Mid-range product,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,False,196,1,1.0,...,9.0,Mid-range product,Least busy,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,False,196,1,1.0,...,9.0,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,False,196,1,1.0,...,9.0,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3226575,360,1,5,12,NaN,True,196,1,0.0,...,9.0,Mid-range product,Regularly busy,Most orders,3,New customer,10.006250,High spender,4.0,Frequent customer
96,1469869,377,3,5,17,3.0,False,196,9,0.0,...,9.0,Mid-range product,Regularly busy,Average orders,3,New customer,8.496552,Low spender,16.5,Regular customer
97,1927023,387,2,4,10,22.0,False,196,3,0.0,...,9.0,Mid-range product,Regularly busy,Most orders,8,New customer,7.396610,Low spender,8.0,Frequent customer
98,858092,420,4,1,19,30.0,False,196,2,0.0,...,9.0,Mid-range product,Regularly busy,Average orders,22,Regular customer,7.387805,Low spender,7.0,Frequent customer


In [24]:
ords_prods_merge.shape

(32400914, 24)

In [25]:
print(ords_prods_merge.dtypes)

order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
new_customer                  bool
product_id                   int64
add_to_cart_order            int64
reordered                  float64
_merge                    category
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
price_range_loc             object
bussiest_days               object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
avg_order                  float64
cost_flag                   object
order_frequency            float64
order_frequency_flag        object
dtype: object


In [26]:
df_merged_large = ords_prods_merge.merge(customers, on = ['user_id'])

In [27]:
df_merged_large.shape

(32400914, 31)

In [28]:
df_merged_large.head(50)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,cost_flag,order_frequency,order_frequency_flag,Gender,STATE,Age,date_joined,Children,Family_status,income
0,2539329,1,1,2,8,NaN,True,196,1,0.0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,False,196,1,1.0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,False,196,1,1.0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,False,196,1,1.0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,False,196,1,1.0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
5,3367565,1,6,2,7,19.0,False,196,1,1.0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
6,550135,1,7,1,9,20.0,False,196,1,1.0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
7,3108588,1,8,1,14,14.0,False,196,2,1.0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
8,2295261,1,9,1,16,0.0,False,196,4,1.0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
9,2550362,1,10,4,8,30.0,False,196,1,1.0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


# Data export

In [29]:
df_merged_large.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers.pkl'))